In [1]:


import ee
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
import numpy as np
from pysheds.grid import Grid
from scipy.ndimage import generic_filter, distance_transform_edt
import matplotlib.pyplot as plt
import os
import time # To handle potential GEE export delays
from dotenv import load_dotenv # Import the library

# --- 0. Load Environment Variables and Initialize Earth Engine ---
load_dotenv()  # Load variables from .env file into environment variables

# Get the Project ID from the environment variable
GCP_PROJECT_ID = os.getenv('GCP_PROJECT_ID')

if not GCP_PROJECT_ID:
    print("Error: GCP_PROJECT_ID not found in .env file or environment variables.")
    print("Please create a .env file with GCP_PROJECT_ID='your-project-id'")
    exit()

try:
    # Pass the project ID to Initialize()
    ee.Initialize(project=GCP_PROJECT_ID)
except ee.EEException as e: # Catch specific EE exceptions
    if 'Please specify a project' in str(e) or 'no project found' in str(e):
        print(f"Initialization with project ID '{GCP_PROJECT_ID}' failed: {e}")
        print("This might be due to an incorrect project ID or insufficient permissions.")
        print("Attempting authentication (this might open a browser window)...")
        try:
            ee.Authenticate() # This will guide you through web authentication
            # After authentication, try initializing again with the project ID
            ee.Initialize(project=GCP_PROJECT_ID)
        except Exception as auth_e:
            print(f"Authentication or subsequent initialization failed: {auth_e}")
            print("Please ensure your .env file has the correct GCP_PROJECT_ID and you have authenticated successfully.")
            exit()
    else: # Other EEException
        print(f"An Earth Engine exception occurred during initialization: {e}")
        exit()
except Exception as e: # Catch any other general exceptions during initialization
    print(f"A general exception occurred during Earth Engine initialization: {e}")
    exit()


print(f"Google Earth Engine initialized with project: {GCP_PROJECT_ID}")

# --- 1. Define AOI and File Paths ---
output_dir = 'output_data_gee/' # Create this directory
os.makedirs(output_dir, exist_ok=True)

# Define your broader AOI (e.g., around São Francisco do Guaporé / Seringueiras)
# GEE uses [lon, lat] order for coordinates
# These are illustrative coordinates for a broader region
aoi_coordinates_gee = [
    [-63.8, -13.0],  # Bottom-left (lon, lat)
    [-62.8, -13.0],  # Bottom-right
    [-62.8, -12.0],  # Top-right
    [-63.8, -12.0],  # Top-left
    [-63.8, -13.0]   # Close polygon
]
aoi_geometry_gee = ee.Geometry.Polygon(aoi_coordinates_gee)

gee_dem_path = os.path.join(output_dir, 'gee_srtm_aoi.tif') # Path to save downloaded DEM

# --- 2. Acquire DEM from Google Earth Engine ---
print("Acquiring SRTM DEM from Google Earth Engine...")
# SRTM 1 Arc-Second Global, Version 3
srtm = ee.Image('USGS/SRTMGL1_003')

# Clip the SRTM image to your AOI
srtm_aoi = srtm.clip(aoi_geometry_gee)

# Get information about the projection of the SRTM image to use for export
# We'll export in its native projection to start
projection_info = srtm.projection().getInfo()
crs_gee = projection_info['crs']
transform_gee = projection_info['transform'] # This is affine transform [scaleX, shearX, translateX, shearY, scaleY, translateY]

print(f"GEE SRTM native CRS: {crs_gee}")
print(f"GEE SRTM native transform: {transform_gee}")

# Export the clipped DEM to Google Drive (or directly download if small enough)
# For larger areas, exporting to Drive is more robust.
# We'll attempt direct download here using getDownloadURL for simplicity,
# but this might time out or fail for very large AOIs.
task_config = {
    'image': srtm_aoi.select('elevation'), # Select the elevation band
    'description': 'SRTM_AOI_Export',
    'scale': 30, # Approximate scale of SRTM 1-arcsec in meters
    'region': aoi_geometry_gee,
    'fileFormat': 'GeoTIFF',
    # 'crs': crs_gee, # Export in native projection
    # 'crsTransform': transform_gee # Using scale is often simpler
}

# --- Option A: Direct Download (may fail for large AOIs or return ZIP) ---
download_success = False
actual_dem_path_for_pysheds = gee_dem_path # Assume it's the direct path initially

try:
    print("Attempting direct download from GEE...")
    if not isinstance(srtm_aoi, ee.Image):
        print("Error: srtm_aoi is not an ee.Image object.")
        raise Exception("Invalid GEE Image object for download")
    
    image_to_download = srtm_aoi.select('elevation')

    download_url = image_to_download.getDownloadURL({
        'scale': 30,
        'region': aoi_geometry_gee,
        'format': 'GEO_TIFF' # Request GeoTIFF
    })
    
    print(f"Generated download URL: {download_url[:100]}...")

    import requests
    import shutil
    import zipfile # For handling ZIP files

    with requests.Session() as session:
        response = session.get(download_url, stream=True, timeout=300)
    
    print(f"Download response status code: {response.status_code}")
    content_type = response.headers.get('Content-Type', '').lower()
    print(f"Download response content-type: {content_type}")

    if response.status_code == 200:
        temp_download_path = gee_dem_path + ".download" # Download to a temp name
        with open(temp_download_path, 'wb') as f:
            shutil.copyfileobj(response.raw, f)
        print(f"SRTM DEM for AOI potentially downloaded to {temp_download_path}")

        # Check if it's a ZIP file
        if 'zip' in content_type or zipfile.is_zipfile(temp_download_path):
            print("Downloaded file is a ZIP archive. Extracting...")
            with zipfile.ZipFile(temp_download_path, 'r') as zip_ref:
                # Find the .tif file within the zip
                tif_files_in_zip = [name for name in zip_ref.namelist() if name.lower().endswith('.tif')]
                if tif_files_in_zip:
                    extracted_tif_name = tif_files_in_zip[0] # Assume the first .tif is the one we want
                    zip_ref.extract(extracted_tif_name, path=output_dir)
                    # Rename the extracted file to our expected gee_dem_path
                    # or ensure actual_dem_path_for_pysheds points to it
                    extracted_file_full_path = os.path.join(output_dir, extracted_tif_name)
                    if os.path.exists(gee_dem_path) and gee_dem_path != extracted_file_full_path:
                         os.remove(gee_dem_path) # Remove if it was a placeholder or previous bad download
                    if gee_dem_path != extracted_file_full_path:
                        os.rename(extracted_file_full_path, gee_dem_path)

                    actual_dem_path_for_pysheds = gee_dem_path
                    print(f"Extracted '{extracted_tif_name}' to '{actual_dem_path_for_pysheds}'")
                else:
                    print("Error: ZIP file downloaded, but no .tif file found inside.")
                    download_success = False
            os.remove(temp_download_path) # Clean up the zip file
        else:
            # Not a zip, assume it's the direct GeoTIFF, rename it
            if os.path.exists(gee_dem_path) and gee_dem_path != temp_download_path:
                 os.remove(gee_dem_path)
            os.rename(temp_download_path, gee_dem_path)
            actual_dem_path_for_pysheds = gee_dem_path
        
        # Verify the final DEM file (either extracted or directly downloaded)
        if os.path.exists(actual_dem_path_for_pysheds):
            try:
                with rasterio.open(actual_dem_path_for_pysheds) as test_ds:
                    print(f"Successfully opened final GeoTIFF: {test_ds.count} band(s), {test_ds.width}x{test_ds.height} pixels.")
                download_success = True
            except rasterio.errors.RasterioIOError:
                print(f"Error: Final file {actual_dem_path_for_pysheds} is not a valid GeoTIFF.")
                if os.path.exists(actual_dem_path_for_pysheds): os.remove(actual_dem_path_for_pysheds)
                download_success = False
        else:
            print(f"Error: Expected DEM file {actual_dem_path_for_pysheds} not found after download/extraction attempt.")
            download_success = False
            
    else: # response.status_code != 200
        print(f"Failed to download directly. Status code: {response.status_code}")
        try:
            print(f"Error response from GEE: {response.content.decode('utf-8', errors='ignore')[:500]}")
        except: pass
        download_success = False

    if not download_success:
        print("Consider exporting to Google Drive for larger or problematic AOIs (Option B).")
        raise Exception("Direct download failed or produced an invalid file.")

except Exception as e_direct_download:
    # ... (rest of the Google Drive export fallback logic as before) ...
    print(f"Direct download from GEE failed or not chosen: {e_direct_download}")
    print("If the error was 'Image.clip: Output of image computation is too large' or similar, your AOI is too big for direct download.")
    print("Proceeding with Google Drive export option (ensure relevant code block is uncommented)...")
    # --- Option B: Export to Google Drive (more robust for larger areas) ---
    # Ensure the task_config is correctly defined if using this block
    task_config_drive = {
        'image': srtm_aoi.select('elevation'),
        'description': 'SRTM_AOI_Export_Drive',
        'scale': 30,
        'region': aoi_geometry_gee,
        'fileFormat': 'GeoTIFF',
        'folder': 'GEE_Exports',
        'fileNamePrefix': 'srtm_aoi_rondonia'
    }
    task = ee.batch.Export.image.toDrive(**task_config_drive)
    task.start()
    print(f"Exporting SRTM DEM for AOI to Google Drive. Task ID: {task.id}")
    print("Please monitor the 'Tasks' tab in the GEE Code Editor or use 'task.status()' to check progress.")
    print(f"Once complete, download '{task_config_drive['fileNamePrefix']}.tif' from your 'GEE_Exports' Drive folder, ensure it's named '{os.path.basename(gee_dem_path)}' in '{os.path.dirname(gee_dem_path)}', and re-run the script from step 3 (after commenting out the GEE download/export sections).")
    exit()


# --- Check if DEM was downloaded and processed successfully ---
if not download_success or not os.path.exists(actual_dem_path_for_pysheds):
    print(f"Error: DEM file {actual_dem_path_for_pysheds} not found or invalid. Please ensure it was downloaded and extracted correctly.")
    exit()

# --- 3. Hydrological Analysis with PySheds ---
print(f"Starting hydrological analysis with PySheds using: {actual_dem_path_for_pysheds}")
grid = Grid.from_raster(actual_dem_path_for_pysheds, data_name='dem')
# Now grid.dem should be the Raster object for your initial DEM

# Fill depressions
print("Filling depressions...")
# Pass the Raster object grid.dem to the 'dem' parameter
grid.fill_depressions(dem=grid.dem, out_name='flooded_dem')
# Now grid.flooded_dem is the Raster object for the filled DEM

# Calculate flow direction
print("Calculating flow direction...")
# Pass the Raster object grid.flooded_dem to the 'dem' parameter
grid.flowdir(dem=grid.flooded_dem, out_name='fdir')
# Now grid.fdir is the Raster object for flow direction

# Calculate flow accumulation
print("Calculating flow accumulation...")
# For accumulation, using the string name with 'data' often works,
# as it might do the lookup internally. If this fails, try data=grid.fdir
grid.accumulation(data='fdir', out_name='acc')
# Now grid.acc is the Raster object for flow accumulation

# Extract stream network
stream_threshold = 1000
print(f"Extracting stream network with threshold: {stream_threshold}...")
# Pass the Raster objects grid.fdir and grid.acc
grid.extract_river_network(fdir=grid.fdir, acc=grid.acc, threshold=stream_threshold, out_name='streams')
# Now grid.streams is the Raster object for the stream network

streams_raster = grid.streams.astype(np.uint8)

# Save the streams raster
streams_path = os.path.join(output_dir, 'streams_gee.tif')
with rasterio.open(actual_dem_path_for_pysheds) as src_meta_provider:
    profile = src_meta_provider.profile.copy()
    profile.update(dtype=rasterio.uint8, count=1, compress='lzw')
    with rasterio.open(streams_path, 'w', **profile) as dst:
        dst.write(streams_raster, 1)
print(f"Streams raster saved to {streams_path}")

# --- 4. Identify Interfluve Zones (remains largely the same) ---

# Method 1: Distance from Streams
print("Calculating distance from streams...")
binary_streams = (streams_raster > 0).astype(np.uint8)
distance_to_streams = distance_transform_edt(1 - binary_streams)
distance_interfluve_threshold_pixels = 15  # EXAMPLE
interfluves_by_distance = (distance_to_streams > distance_interfluve_threshold_pixels).astype(np.uint8)

interfluves_dist_path = os.path.join(output_dir, 'interfluves_by_distance_gee.tif')
with rasterio.open(streams_path) as src_meta_provider:
    profile = src_meta_provider.profile.copy()
    with rasterio.open(interfluves_dist_path, 'w', **profile) as dst:
        dst.write(interfluves_by_distance, 1)
print(f"Interfluves by distance saved to {interfluves_dist_path}")


# Method 2: Topographic Position Index (TPI)
print("Calculating TPI...")
with rasterio.open(gee_dem_path) as src:
    dem_array = src.read(1).astype(np.float32) # Ensure float for calculations
    profile = src.profile.copy()

kernel_size = 9  # EXAMPLE
pad_width = kernel_size // 2
# Handle NaN values in DEM if any before padding/filtering, e.g., by interpolating or setting to a value
dem_array_no_nan = np.nan_to_num(dem_array, nan=np.nanmean(dem_array)) # Example: replace NaN with mean

dem_padded = np.pad(dem_array_no_nan, pad_width, mode='reflect')

def mean_filter_nan_aware(arr): # TPI calculation needs to be robust to NaNs if present
    valid_arr = arr[~np.isnan(arr)]
    if valid_arr.size == 0:
        return np.nan
    return np.mean(valid_arr)

mean_elevation_neighborhood = generic_filter(
    dem_padded,
    mean_filter_nan_aware, # Use NaN-aware mean
    size=kernel_size,
    mode='constant',
    cval=np.nan
)
mean_elevation_neighborhood = mean_elevation_neighborhood[pad_width:-pad_width, pad_width:-pad_width]

tpi = dem_array - mean_elevation_neighborhood # Original dem_array might have NaNs, so TPI will too

tpi_interfluve_threshold = 0.5  # EXAMPLE
# Handle NaNs in TPI before comparison
interfluves_by_tpi = np.where(np.isnan(tpi), 0, (tpi > tpi_interfluve_threshold)).astype(np.uint8)


tpi_path = os.path.join(output_dir, 'tpi_gee.tif')
interfluves_tpi_path = os.path.join(output_dir, 'interfluves_by_tpi_gee.tif')
tpi_profile = profile.copy()
tpi_profile.update(dtype=rasterio.float32, compress='lzw')
with rasterio.open(tpi_path, 'w', **tpi_profile) as dst:
    dst.write(tpi.astype(rasterio.float32), 1) # Save TPI with potential NaNs

interfluve_tpi_profile = profile.copy()
interfluve_tpi_profile.update(dtype=rasterio.uint8, compress='lzw')
with rasterio.open(interfluves_tpi_path, 'w', **interfluve_tpi_profile) as dst:
    dst.write(interfluves_by_tpi, 1)
print(f"TPI saved to {tpi_path}")
print(f"Interfluves by TPI saved to {interfluves_tpi_path}")


# Method 3: Combine Distance and TPI
print("Combining distance and TPI methods for interfluves...")
if interfluves_by_distance.shape == interfluves_by_tpi.shape:
    combined_interfluves = (interfluves_by_distance & interfluves_by_tpi).astype(np.uint8)
    combined_interfluves_path = os.path.join(output_dir, 'combined_interfluves_gee.tif')
    with rasterio.open(streams_path) as src_meta_provider:
        profile = src_meta_provider.profile.copy()
        with rasterio.open(combined_interfluves_path, 'w', **profile) as dst:
            dst.write(combined_interfluves, 1)
    print(f"Combined interfluves saved to {combined_interfluves_path}")
else:
    print("Shapes of distance and TPI interfluve arrays do not match. Skipping combination.")

print("Processing complete. Check the output_data_gee directory.")



Google Earth Engine initialized with project: amazon-460111
Acquiring SRTM DEM from Google Earth Engine...
GEE SRTM native CRS: EPSG:4326
GEE SRTM native transform: [0.0002777777777777778, 0, -180.0001388888889, 0, -0.0002777777777777778, 60.00013888888889]
Attempting direct download from GEE...
Generated download URL: https://earthengine.googleapis.com/v1/projects/amazon-460111/thumbnails/73b4278548252f6ad99ef8280da5...
Download response status code: 200
Download response content-type: image/tiff
SRTM DEM for AOI potentially downloaded to output_data_gee/gee_srtm_aoi.tif.download
Successfully opened final GeoTIFF: 1 band(s), 3711x3713 pixels.
Starting hydrological analysis with PySheds using: output_data_gee/gee_srtm_aoi.tif
Filling depressions...


AttributeError: 'sGrid' object has no attribute 'dem'

In [2]:

# --- 3. Hydrological Analysis with PySheds ---
print(f"Starting hydrological analysis with PySheds using: {actual_dem_path_for_pysheds}")
grid = Grid.from_raster(actual_dem_path_for_pysheds, data_name='dem')
# Now grid.dem should be the Raster object for your initial DEM

# Fill depressions
print("Filling depressions...")
# Pass the Raster object grid.dem to the 'dem' parameter
grid.fill_depressions(dem=grid.dem, out_name='flooded_dem')
# Now grid.flooded_dem is the Raster object for the filled DEM

# Calculate flow direction
print("Calculating flow direction...")
# Pass the Raster object grid.flooded_dem to the 'dem' parameter
grid.flowdir(dem=grid.flooded_dem, out_name='fdir')
# Now grid.fdir is the Raster object for flow direction

# Calculate flow accumulation
print("Calculating flow accumulation...")
# For accumulation, using the string name with 'data' often works,
# as it might do the lookup internally. If this fails, try data=grid.fdir
grid.accumulation(data='fdir', out_name='acc')
# Now grid.acc is the Raster object for flow accumulation

# Extract stream network
stream_threshold = 1000
print(f"Extracting stream network with threshold: {stream_threshold}...")
# Pass the Raster objects grid.fdir and grid.acc
grid.extract_river_network(fdir=grid.fdir, acc=grid.acc, threshold=stream_threshold, out_name='streams')
# Now grid.streams is the Raster object for the stream network

streams_raster = grid.streams.astype(np.uint8)

# Save the streams raster
streams_path = os.path.join(output_dir, 'streams_gee.tif')
with rasterio.open(actual_dem_path_for_pysheds) as src_meta_provider:
    profile = src_meta_provider.profile.copy()
    profile.update(dtype=rasterio.uint8, count=1, compress='lzw')
    with rasterio.open(streams_path, 'w', **profile) as dst:
        dst.write(streams_raster, 1)
print(f"Streams raster saved to {streams_path}")

# --- 4. Identify Interfluve Zones (remains largely the same) ---

# Method 1: Distance from Streams
print("Calculating distance from streams...")
binary_streams = (streams_raster > 0).astype(np.uint8)
distance_to_streams = distance_transform_edt(1 - binary_streams)
distance_interfluve_threshold_pixels = 15  # EXAMPLE
interfluves_by_distance = (distance_to_streams > distance_interfluve_threshold_pixels).astype(np.uint8)

interfluves_dist_path = os.path.join(output_dir, 'interfluves_by_distance_gee.tif')
with rasterio.open(streams_path) as src_meta_provider:
    profile = src_meta_provider.profile.copy()
    with rasterio.open(interfluves_dist_path, 'w', **profile) as dst:
        dst.write(interfluves_by_distance, 1)
print(f"Interfluves by distance saved to {interfluves_dist_path}")


# Method 2: Topographic Position Index (TPI)
print("Calculating TPI...")
with rasterio.open(gee_dem_path) as src:
    dem_array = src.read(1).astype(np.float32) # Ensure float for calculations
    profile = src.profile.copy()

kernel_size = 9  # EXAMPLE
pad_width = kernel_size // 2
# Handle NaN values in DEM if any before padding/filtering, e.g., by interpolating or setting to a value
dem_array_no_nan = np.nan_to_num(dem_array, nan=np.nanmean(dem_array)) # Example: replace NaN with mean

dem_padded = np.pad(dem_array_no_nan, pad_width, mode='reflect')

def mean_filter_nan_aware(arr): # TPI calculation needs to be robust to NaNs if present
    valid_arr = arr[~np.isnan(arr)]
    if valid_arr.size == 0:
        return np.nan
    return np.mean(valid_arr)

mean_elevation_neighborhood = generic_filter(
    dem_padded,
    mean_filter_nan_aware, # Use NaN-aware mean
    size=kernel_size,
    mode='constant',
    cval=np.nan
)
mean_elevation_neighborhood = mean_elevation_neighborhood[pad_width:-pad_width, pad_width:-pad_width]

tpi = dem_array - mean_elevation_neighborhood # Original dem_array might have NaNs, so TPI will too

tpi_interfluve_threshold = 0.5  # EXAMPLE
# Handle NaNs in TPI before comparison
interfluves_by_tpi = np.where(np.isnan(tpi), 0, (tpi > tpi_interfluve_threshold)).astype(np.uint8)


tpi_path = os.path.join(output_dir, 'tpi_gee.tif')
interfluves_tpi_path = os.path.join(output_dir, 'interfluves_by_tpi_gee.tif')
tpi_profile = profile.copy()
tpi_profile.update(dtype=rasterio.float32, compress='lzw')
with rasterio.open(tpi_path, 'w', **tpi_profile) as dst:
    dst.write(tpi.astype(rasterio.float32), 1) # Save TPI with potential NaNs

interfluve_tpi_profile = profile.copy()
interfluve_tpi_profile.update(dtype=rasterio.uint8, compress='lzw')
with rasterio.open(interfluves_tpi_path, 'w', **interfluve_tpi_profile) as dst:
    dst.write(interfluves_by_tpi, 1)
print(f"TPI saved to {tpi_path}")
print(f"Interfluves by TPI saved to {interfluves_tpi_path}")


# Method 3: Combine Distance and TPI
print("Combining distance and TPI methods for interfluves...")
if interfluves_by_distance.shape == interfluves_by_tpi.shape:
    combined_interfluves = (interfluves_by_distance & interfluves_by_tpi).astype(np.uint8)
    combined_interfluves_path = os.path.join(output_dir, 'combined_interfluves_gee.tif')
    with rasterio.open(streams_path) as src_meta_provider:
        profile = src_meta_provider.profile.copy()
        with rasterio.open(combined_interfluves_path, 'w', **profile) as dst:
            dst.write(combined_interfluves, 1)
    print(f"Combined interfluves saved to {combined_interfluves_path}")
else:
    print("Shapes of distance and TPI interfluve arrays do not match. Skipping combination.")

print("Processing complete. Check the output_data_gee directory.")

Starting hydrological analysis with PySheds using: output_data_gee/gee_srtm_aoi.tif
Filling depressions...


AttributeError: 'sGrid' object has no attribute 'dem'

In [4]:
# --- 3. Hydrological Analysis with PySheds ---
print(f"Starting hydrological analysis with PySheds using: {actual_dem_path_for_pysheds}")
# This registers a grid under the name 'dem'
grid = Grid.from_raster(actual_dem_path_for_pysheds, data_name='dem')

# --- PySheds Debugging: Inspect the grid object ---
print(f"Type of grid: {type(grid)}")
print(f"Is 'dem' an attribute? {'dem' in dir(grid)}")
if hasattr(grid, '_grids'):
    print(f"Internal grid keys: {list(grid._grids.keys())}")
    if 'dem' in grid._grids:
        print(f"Type of internal 'dem' grid: {type(grid._grids['dem'])}")
else:
    print("No '_grids' attribute found.")
# --- End PySheds Debugging ---


# Fill depressions
# Most PySheds functions expect the *name* of the input grid via the 'data' parameter.
# The output grid is then registered under 'out_name'.
print("Filling depressions...")
grid.fill_depressions(data='dem', out_name='flooded_dem') # Using 'data' keyword

# Calculate flow direction
# The input is 'flooded_dem' (the out_name from the previous step)
print("Calculating flow direction...")
grid.flowdir(data='flooded_dem', out_name='fdir') # Using 'data' keyword

# Calculate flow accumulation
# The input is 'fdir'
print("Calculating flow accumulation...")
grid.accumulation(data='fdir', out_name='acc') # Using 'data' keyword

# Extract stream network
stream_threshold = 1000
print(f"Extracting stream network with threshold: {stream_threshold}...")
# This function also typically takes the *names* of the flow direction and accumulation grids.
grid.extract_river_network(fdir='fdir', acc='acc', threshold=stream_threshold, out_name='streams')

# IMPORTANT: Accessing the output raster
# After the above operations, PySheds typically makes the resulting Raster object
# available as an attribute on the grid, using its 'out_name'.
# So, grid.streams should now exist if extract_river_network completed successfully
# and registered 'streams'.
try:
    streams_raster = grid.streams.astype(np.uint8)

    # Save the streams raster
    streams_path = os.path.join(output_dir, 'streams_gee.tif')
    with rasterio.open(actual_dem_path_for_pysheds) as src_meta_provider:
        profile = src_meta_provider.profile.copy()
        profile.update(dtype=rasterio.uint8, count=1, compress='lzw')
        with rasterio.open(streams_path, 'w', **profile) as dst:
            dst.write(streams_raster, 1)
    print(f"Streams raster saved to {streams_path}")

except AttributeError as e:
    print(f"Error accessing resulting stream raster: {e}")
    print("This might mean the 'streams' grid was not set as an attribute after 'extract_river_network'.")
    print("Check if 'streams' is in grid._grids.keys() if the processing seemed to complete.")
    if hasattr(grid, '_grids'):
        print(f"Available internal grids: {list(grid._grids.keys())}")
except Exception as e:
    print(f"An unexpected error occurred during stream processing or saving: {e}")


# --- 4. Identify Interfluve Zones (remains largely the same) ---
# This section assumes streams_raster was successfully created.
# If the try-except block above fails, this part will also fail.

if 'streams_raster' in locals(): # Check if streams_raster was defined
    # Method 1: Distance from Streams
    print("Calculating distance from streams...")
    binary_streams = (streams_raster > 0).astype(np.uint8)
    # ... (rest of your interfluve code) ...
else:
    print("Skipping interfluve analysis because 'streams_raster' was not created.")


Starting hydrological analysis with PySheds using: output_data_gee/gee_srtm_aoi.tif
Type of grid: <class 'pysheds.sgrid.sGrid'>
Is 'dem' an attribute? False
No '_grids' attribute found.
Filling depressions...


TypeError: sGrid.fill_depressions() missing 1 required positional argument: 'dem'

In [5]:
import logging
import pysheds
from pysheds.grid import Grid # Explicitly importing Grid
import numpy as np # For any array operations if needed
import os # For file path joining if needed, though actual_dem_path_for_pysheds is assumed

# --- Setup Logging ---
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger()

# --- Assumed to be predefined in your Jupyter environment ---
# output_dir = 'output_data_gee/'
# actual_dem_path_for_pysheds = os.path.join(output_dir, 'gee_srtm_aoi.tif')
# print(f"Using DEM path: {actual_dem_path_for_pysheds}")
# if not os.path.exists(actual_dem_path_for_pysheds):
#     print(f"CRITICAL ERROR: DEM file {actual_dem_path_for_pysheds} not found. Testing cannot proceed.")
#     # exit() # or raise an error in a script

logger.info(f"Starting PySheds sGrid focused testing with DEM: {actual_dem_path_for_pysheds}")

# --- Test 1: Grid Initialization and Loading with from_raster ---
logger.info("--- Test 1: Grid.from_raster() ---")
grid_instance = None
try:
    # Attempt 1a: Standard data_name='dem'
    logger.info("Test 1a: Grid.from_raster with data_name='dem'")
    grid_instance = Grid.from_raster(actual_dem_path_for_pysheds, data_name='dem')
    logger.info(f"Grid object type: {type(grid_instance)}")
    logger.info(f"Attributes of grid_instance: {dir(grid_instance)}")
    logger.info(f"grid_instance.__dict__: {grid_instance.__dict__}")

    if hasattr(grid_instance, 'dem'):
        logger.info("SUCCESS: grid_instance.dem attribute exists.")
        logger.info(f"Type of grid_instance.dem: {type(grid_instance.dem)}")
    else:
        logger.warning("grid_instance.dem attribute DOES NOT exist after data_name='dem'.")

    # Attempt 1b: data_name='DEM' (uppercase, based on some sGrid discussions)
    logger.info("Test 1b: Grid.from_raster with data_name='DEM'")
    grid_instance_upper = Grid.from_raster(actual_dem_path_for_pysheds, data_name='DEM')
    logger.info(f"Grid object type (uppercase test): {type(grid_instance_upper)}")
    if hasattr(grid_instance_upper, 'DEM'):
        logger.info("SUCCESS: grid_instance_upper.DEM attribute exists.")
        logger.info(f"Type of grid_instance_upper.DEM: {type(grid_instance_upper.DEM)}")
        grid_instance = grid_instance_upper # Use this if successful
        logger.info("Switching to grid_instance_upper for subsequent tests if .DEM exists.")
    elif hasattr(grid_instance_upper, 'dem'): # Check lowercase 'dem' just in case
        logger.info("grid_instance_upper.dem (lowercase) attribute exists even with data_name='DEM'.")
        logger.info(f"Type of grid_instance_upper.dem: {type(grid_instance_upper.dem)}")
        if not hasattr(grid_instance, 'dem'): # If initial 'dem' attempt failed but this one (somehow) created 'dem'
            grid_instance = grid_instance_upper
            logger.info("Switching to grid_instance_upper as it created a 'dem' attribute.")
    else:
        logger.warning("grid_instance_upper.DEM (and .dem) attribute DOES NOT exist after data_name='DEM'.")

except Exception as e:
    logger.error(f"Error during Test 1 (Grid.from_raster): {e}", exc_info=True)

# --- Test 2: Using grid.read_raster() ---
# sGrid might primarily work by having an empty Grid and then using read_raster
logger.info("--- Test 2: grid.read_raster() ---")
dem_raster_object = None
grid_for_read_raster = Grid() # Create a fresh, empty grid
logger.info(f"Created empty grid_for_read_raster: {type(grid_for_read_raster)}")
try:
    logger.info("Test 2a: grid.read_raster with data_name='dem_read'")
    # read_raster usually registers the name AND returns the object.
    dem_raster_object = grid_for_read_raster.read_raster(actual_dem_path_for_pysheds, data_name='dem_read')
    logger.info(f"grid.read_raster returned object of type: {type(dem_raster_object)}")
    if isinstance(dem_raster_object, pysheds.grid.Raster): # Check if it's a Raster object
        logger.info("SUCCESS: grid.read_raster returned a pysheds.grid.Raster object.")
    else:
        logger.warning("grid.read_raster did not return a pysheds.grid.Raster object.")

    # Check if 'dem_read' is now an attribute or accessible
    if hasattr(grid_for_read_raster, 'dem_read'):
        logger.info("grid_for_read_raster.dem_read attribute exists.")
    else:
        logger.warning("grid_for_read_raster.dem_read attribute DOES NOT exist.")
    
    # Verify internal registration (if _grids exists, which it didn't before, but good to check)
    if hasattr(grid_for_read_raster, '_grids') and 'dem_read' in grid_for_read_raster._grids:
        logger.info("'dem_read' is registered in grid_for_read_raster._grids.")
    elif hasattr(grid_for_read_raster, '_grids'):
        logger.warning(f"'dem_read' NOT in grid_for_read_raster._grids. Keys: {list(grid_for_read_raster._grids.keys())}")
    else:
        logger.info("grid_for_read_raster still does not have a '_grids' attribute after read_raster.")


except Exception as e:
    logger.error(f"Error during Test 2 (grid.read_raster): {e}", exc_info=True)


# --- Test 3: Calling fill_depressions ---
logger.info("--- Test 3: Calling fill_depressions ---")

# Scenario 3a: Using grid_instance from Test 1 (if grid.dem or grid.DEM was found)
if grid_instance and (hasattr(grid_instance, 'dem') or hasattr(grid_instance, 'DEM')):
    target_dem_attr_name = 'DEM' if hasattr(grid_instance, 'DEM') else 'dem'
    logger.info(f"Test 3a: Using grid_instance from Test 1, with attribute '{target_dem_attr_name}'")
    try:
        dem_to_fill = getattr(grid_instance, target_dem_attr_name)
        logger.info(f"Type of dem_to_fill (from grid_instance.{target_dem_attr_name}): {type(dem_to_fill)}")
        if isinstance(dem_to_fill, pysheds.grid.Raster):
            grid_instance.fill_depressions(dem=dem_to_fill, out_name='flooded_test1')
            logger.info("SUCCESS: grid_instance.fill_depressions completed using the direct attribute.")
            if hasattr(grid_instance, 'flooded_test1'):
                 logger.info("grid_instance.flooded_test1 attribute created.")
            else:
                 logger.warning("grid_instance.flooded_test1 attribute NOT created.")
        else:
            logger.error(f"Attribute grid_instance.{target_dem_attr_name} is not a Raster object. Cannot proceed.")
    except Exception as e:
        logger.error(f"Error during Test 3a (fill_depressions with attribute from Grid.from_raster): {e}", exc_info=True)
else:
    logger.warning("Skipping Test 3a because a usable DEM attribute was not found in grid_instance from Test 1.")


# Scenario 3b: Using dem_raster_object from Test 2 (if grid.read_raster returned a Raster object)
# And using the grid_for_read_raster instance
if dem_raster_object and isinstance(dem_raster_object, pysheds.grid.Raster):
    logger.info("Test 3b: Using dem_raster_object from Test 2 (from grid.read_raster)")
    try:
        # We pass the dem_raster_object to the 'dem' parameter.
        # The fill_depressions method is called on grid_for_read_raster.
        # It should use dem_raster_object as input and register 'flooded_test2' within grid_for_read_raster
        grid_for_read_raster.fill_depressions(dem=dem_raster_object, out_name='flooded_test2')
        logger.info("SUCCESS: grid_for_read_raster.fill_depressions completed using object from read_raster.")
        if hasattr(grid_for_read_raster, 'flooded_test2'):
             logger.info("grid_for_read_raster.flooded_test2 attribute created.")
        else:
             logger.warning("grid_for_read_raster.flooded_test2 attribute NOT created.")
        # Also check if 'flooded_test2' was registered by its name 'dem_read' (the input object's original name)
        if hasattr(grid_for_read_raster, '_grids') and 'dem_read' in grid_for_read_raster._grids:
             logger.info("The original input 'dem_read' is still in _grids.")
        if hasattr(grid_for_read_raster, '_grids') and 'flooded_test2' in grid_for_read_raster._grids:
             logger.info("The output 'flooded_test2' is in _grids.")

    except Exception as e:
        logger.error(f"Error during Test 3b (fill_depressions with object from grid.read_raster): {e}", exc_info=True)
else:
    logger.warning("Skipping Test 3b because dem_raster_object from Test 2 is not a valid Raster object.")

# Scenario 3c: Attempting to use the *name* with fill_depressions, using grid_for_read_raster
# This is based on the idea that read_raster(data_name='dem_read') internally registered it.
# This will likely give the same TypeError as before, but included for completeness.
if hasattr(grid_for_read_raster, 'read_raster'): # Check if grid_for_read_raster is a valid Grid
    logger.info("Test 3c: Using string name 'dem_read' with fill_depressions on grid_for_read_raster")
    try:
        # We need to ensure 'dem_read' was actually registered by read_raster previously for this to have a chance
        # The dem_raster_object itself isn't used here, only its name.
        # This is what failed before, but let's see if the context of read_raster changes it.
        grid_for_read_raster.fill_depressions(dem='dem_read', out_name='flooded_test3')
        logger.info("SUCCESS (unexpectedly?): grid_for_read_raster.fill_depressions completed using name 'dem_read'.")
        if hasattr(grid_for_read_raster, 'flooded_test3'):
             logger.info("grid_for_read_raster.flooded_test3 attribute created.")
    except TypeError as te:
        if "missing 1 required positional argument: 'dem'" in str(te) or "Data must be a Raster" in str(te):
            logger.warning(f"As expected, Test 3c failed with TypeError: {te}. `dem` parameter needs a Raster object, not a string name for fill_depressions.")
        else:
            logger.error(f"Test 3c failed with an unexpected TypeError: {te}", exc_info=True)
    except Exception as e:
        logger.error(f"Error during Test 3c (fill_depressions with string name 'dem_read'): {e}", exc_info=True)
else:
    logger.warning("Skipping Test 3c as grid_for_read_raster might not be valid.")


logger.info("--- Testing Concluded ---")

2025-05-17 18:01:13,540 - INFO - Starting PySheds sGrid focused testing with DEM: output_data_gee/gee_srtm_aoi.tif
2025-05-17 18:01:13,540 - INFO - --- Test 1: Grid.from_raster() ---
2025-05-17 18:01:13,544 - INFO - Test 1a: Grid.from_raster with data_name='dem'
2025-05-17 18:01:13,555 - WARNING - CPLE_NotSupported in driver GTiff does not support open option DATA_NAME
2025-05-17 18:01:13,840 - INFO - Grid object type: <class 'pysheds.sgrid.sGrid'>
2025-05-17 18:01:13,840 - INFO - Attributes of grid_instance: ['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_d8_accumulation', '_d8_catchment', '_d8_cell_dh', '_d8_cell_distances', '_d8_compute_hand', '_d8_distance_to_ridge', '_d8_

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [6]:
import pysheds
from pysheds.grid import Grid # Make sure Grid is from pysheds.grid
import numpy as np
import rasterio # Still needed for getting metadata for saving outputs
import os

# --- Setup ---
output_dir = 'output_data_gee/'
os.makedirs(output_dir, exist_ok=True)
actual_dem_path_for_pysheds = os.path.join(output_dir, 'gee_srtm_aoi.tif')

if not os.path.exists(actual_dem_path_for_pysheds):
    print(f"CRITICAL: DEM file not found at {actual_dem_path_for_pysheds}")
    # exit() # Or raise error

print(f"Starting hydrological analysis with PySheds (sGrid focus) using: {actual_dem_path_for_pysheds}")

# 1. Create an sGrid instance
grid = Grid()
print(f"Created grid object: {type(grid)}")

# 2. Load the DEM using grid.read_raster()
# This method loads raster properties into the grid object and returns the DEM data as a NumPy array.
# The 'data_name' here is used for internal registration if sGrid supports it,
# but the key is that dem_data will be the NumPy array.
print(f"Loading DEM from {actual_dem_path_for_pysheds}...")
dem_data = grid.read_raster(actual_dem_path_for_pysheds, data_name='dem_loaded') # or just 'dem'
print(f"DEM data loaded. Type: {type(dem_data)}, Shape: {dem_data.shape if isinstance(dem_data, np.ndarray) else 'Not a NumPy array'}")

# Verify grid properties were set (they should be after read_raster)
print(f"Grid shape: {grid.shape}, Grid nodata: {grid.nodata}, Grid affine: {grid.affine}")

# 3. Fill depressions
# Pass the NumPy array dem_data to the 'dem' parameter.
# The output 'flooded_dem' will become an attribute on the grid object (grid.flooded_dem).
print("Filling depressions...")
grid.fill_depressions(dem=dem_data, out_name='flooded_dem')
print(f"Depressions filled. 'grid.flooded_dem' attribute exists: {hasattr(grid, 'flooded_dem')}")
if hasattr(grid, 'flooded_dem'):
    print(f"Type of grid.flooded_dem: {type(grid.flooded_dem)}, Shape: {grid.flooded_dem.shape if isinstance(grid.flooded_dem, np.ndarray) else 'N/A'}")

# 4. Calculate flow direction
# The input is the NumPy array stored in grid.flooded_dem.
print("Calculating flow direction...")
if hasattr(grid, 'flooded_dem'):
    grid.flowdir(dem=grid.flooded_dem, out_name='fdir') # Pass the actual data
    print(f"Flow direction calculated. 'grid.fdir' attribute exists: {hasattr(grid, 'fdir')}")
    if hasattr(grid, 'fdir'):
        print(f"Type of grid.fdir: {type(grid.fdir)}, Shape: {grid.fdir.shape if isinstance(grid.fdir, np.ndarray) else 'N/A'}")
else:
    print("Skipping flow direction as 'flooded_dem' was not found.")

# 5. Calculate flow accumulation
# The input is the NumPy array stored in grid.fdir.
# Note: accumulation typically takes 'data' keyword with the *name* of the flow direction grid,
# but since sGrid seems to prefer direct data, let's check if grid.fdir is the data.
# Alternatively, if 'fdir' was registered as a name by the flowdir function, 'data' keyword might use that.
# Let's assume for consistency 'dem=' takes the array if that's the parameter name.
# Looking at sGrid source or more examples for `accumulation` might be needed.
# For now, let's assume if fill_depressions and flowdir take `dem=` with an array, accumulation might too,
# or it might use a `data=` keyword that looks up the *attribute* `grid.fdir`.
# Most sGrid functions like `accumulation` expect the *name* of the previously computed grid when `data` is used.
# But since `grid.fdir` is now an attribute holding the data array, it's safer to pass the data array.
print("Calculating flow accumulation...")
if hasattr(grid, 'fdir'):
    # Check the signature of grid.accumulation. If it has a `dem=` or `fdir=` param, use grid.fdir.
    # If it primarily uses `data='name'`, then the name 'fdir' should have been registered by the previous step.
    # Let's try passing the data directly to a 'data' param if 'dem' isn't it.
    # The typical call is grid.accumulation(data='fdir', out_name='acc')
    # This implies that 'fdir' (the string) is registered.
    grid.accumulation(data='fdir', out_name='acc') # This relies on 'fdir' being registered by the flowdir out_name
    print(f"Flow accumulation calculated. 'grid.acc' attribute exists: {hasattr(grid, 'acc')}")
    if hasattr(grid, 'acc'):
         print(f"Type of grid.acc: {type(grid.acc)}, Shape: {grid.acc.shape if isinstance(grid.acc, np.ndarray) else 'N/A'}")
else:
    print("Skipping flow accumulation as 'fdir' was not found.")


# 6. Extract stream network
stream_threshold = 1000
print(f"Extracting stream network with threshold: {stream_threshold}...")
if hasattr(grid, 'fdir') and hasattr(grid, 'acc'):
    # Similar to accumulation, extract_river_network usually takes names.
    grid.extract_river_network(fdir='fdir', acc='acc', threshold=stream_threshold, out_name='streams')
    print(f"Stream network extracted. 'grid.streams' attribute exists: {hasattr(grid, 'streams')}")
    if hasattr(grid, 'streams'):
        streams_raster_data = grid.streams # This is the NumPy array
        print(f"Type of grid.streams: {type(streams_raster_data)}, Shape: {streams_raster_data.shape if isinstance(streams_raster_data, np.ndarray) else 'N/A'}")

        # Save the streams raster
        streams_path = os.path.join(output_dir, 'streams_gee.tif')
        # Get metadata from the original DEM for saving
        with rasterio.open(actual_dem_path_for_pysheds) as src_ds:
            profile = src_ds.profile.copy()
            profile.update(dtype=rasterio.uint8, count=1, compress='lzw') # Or streams_raster_data.dtype

        with rasterio.open(streams_path, 'w', **profile) as dst:
            dst.write(streams_raster_data.astype(rasterio.uint8), 1) # Ensure correct dtype for saving
        print(f"Streams raster saved to {streams_path}")
    else:
        print("streams_raster_data could not be created.")
else:
    print("Skipping stream extraction as 'fdir' or 'acc' was not found.")

print("Processing attempt finished.")

2025-05-17 18:03:04,633 - WARNING - CPLE_NotSupported in driver GTiff does not support open option DATA_NAME


Starting hydrological analysis with PySheds (sGrid focus) using: output_data_gee/gee_srtm_aoi.tif
Created grid object: <class 'pysheds.sgrid.sGrid'>
Loading DEM from output_data_gee/gee_srtm_aoi.tif...
DEM data loaded. Type: <class 'pysheds.sview.Raster'>, Shape: (3713, 3711)
Grid shape: (1, 1), Grid nodata: 0, Grid affine: | 1.00, 0.00, 0.00|
| 0.00, 1.00, 0.00|
| 0.00, 0.00, 1.00|
Filling depressions...
Depressions filled. 'grid.flooded_dem' attribute exists: False
Calculating flow direction...
Skipping flow direction as 'flooded_dem' was not found.
Calculating flow accumulation...
Skipping flow accumulation as 'fdir' was not found.
Extracting stream network with threshold: 1000...
Skipping stream extraction as 'fdir' or 'acc' was not found.
Processing attempt finished.


In [7]:
import pysheds
from pysheds.grid import Grid
import numpy as np
import rasterio
import os

# --- Setup ---
output_dir = 'output_data_gee/'
os.makedirs(output_dir, exist_ok=True)
actual_dem_path_for_pysheds = os.path.join(output_dir, 'gee_srtm_aoi.tif')

if not os.path.exists(actual_dem_path_for_pysheds):
    print(f"CRITICAL: DEM file not found at {actual_dem_path_for_pysheds}")
    # exit()

print(f"Starting hydrological analysis with PySheds (sGrid refined) using: {actual_dem_path_for_pysheds}")

# 1. Create an sGrid instance
grid = Grid()
print(f"Created grid object: {type(grid)}")

# 2. Load DEM. grid.read_raster configures the grid instance and returns a RasterView object.
print(f"Loading DEM from {actual_dem_path_for_pysheds}...")
# The data_name here is for internal registration if sGrid uses it.
dem_raster_view = grid.read_raster(actual_dem_path_for_pysheds, data_name='dem_initial')
print(f"DEM RasterView loaded. Type: {type(dem_raster_view)}")
print(f"Grid properties after read_raster: Shape: {grid.shape}, Affine: {grid.affine}, Nodata: {grid.nodata}")

# 3. Fill depressions
# Pass the RasterView object. Expect a new RasterView object as return.
print("Filling depressions...")
# 'out_name' registers 'flooded_dem' internally.
flooded_dem_raster_view = grid.fill_depressions(dem=dem_raster_view, out_name='flooded_dem')
print(f"Depressions filled. Returned type: {type(flooded_dem_raster_view)}")
# Check if 'flooded_dem' can be accessed via grid.view() or if it became an attribute
print(f"Has 'flooded_dem' attribute on grid? {hasattr(grid, 'flooded_dem')}")
try:
    test_view = grid.view('flooded_dem') # Common way to access sGrid layers by name
    print(f"grid.view('flooded_dem') accessible, type: {type(test_view)}")
except KeyError:
    print("grid.view('flooded_dem') not accessible (KeyError).")


# 4. Calculate flow direction
# Pass the returned RasterView object from the previous step.
print("Calculating flow direction...")
if flooded_dem_raster_view is not None and isinstance(flooded_dem_raster_view, pysheds.sview.Raster):
    # 'out_name' registers 'fdir' internally.
    fdir_raster_view = grid.flowdir(dem=flooded_dem_raster_view, out_name='fdir')
    print(f"Flow direction calculated. Returned type: {type(fdir_raster_view)}")
    print(f"Has 'fdir' attribute on grid? {hasattr(grid, 'fdir')}")
    try:
        test_fdir_view = grid.view('fdir')
        print(f"grid.view('fdir') accessible, type: {type(test_fdir_view)}")
    except KeyError:
        print("grid.view('fdir') not accessible (KeyError).")
else:
    fdir_raster_view = None # Ensure it's defined for the next step's check
    print("Skipping flow direction as flooded_dem_raster_view is not a valid RasterView.")


# 5. Calculate flow accumulation
# This function typically takes the *name* of the flow direction grid ('fdir'),
# which should have been registered by out_name='fdir' in the previous step.
print("Calculating flow accumulation...")
if fdir_raster_view is not None: # Check if flowdir step was successful
    # 'out_name' registers 'acc' internally.
    # No direct RasterView is returned by accumulation, it modifies grid state.
    # Or it might return one too, let's be safe:
    acc_maybe_raster_view = grid.accumulation(data='fdir', out_name='acc')
    print(f"Flow accumulation potentially calculated. Returned type by accumulation: {type(acc_maybe_raster_view)}")
    print(f"Has 'acc' attribute on grid? {hasattr(grid, 'acc')}")
    try:
        acc_raster_view_from_grid = grid.view('acc') # This is the typical way to get it
        print(f"grid.view('acc') accessible, type: {type(acc_raster_view_from_grid)}")
    except KeyError:
        acc_raster_view_from_grid = None
        print("grid.view('acc') not accessible (KeyError).")
else:
    acc_raster_view_from_grid = None
    print("Skipping flow accumulation as fdir_raster_view was not created.")

# 6. Extract stream network
stream_threshold = 1000
print(f"Extracting stream network with threshold: {stream_threshold}...")
streams_raster_view = None # Initialize
if acc_raster_view_from_grid is not None: # Check if accumulation was successful
    # 'out_name' registers 'streams' internally.
    # This also likely returns a RasterView or updates grid state.
    streams_maybe_raster_view = grid.extract_river_network(fdir='fdir', acc='acc', threshold=stream_threshold, out_name='streams')
    print(f"Stream network extraction potentially run. Returned type: {type(streams_maybe_raster_view)}")
    print(f"Has 'streams' attribute on grid? {hasattr(grid, 'streams')}")
    try:
        streams_raster_view = grid.view('streams') # The data to save
        print(f"grid.view('streams') accessible, type: {type(streams_raster_view)}")

        # To save, we need the NumPy array from the RasterView object
        # Assuming the RasterView object behaves like a NumPy array or has a .to_numpy() or similar,
        # or can be directly passed to rasterio.write if it's just a masked array.
        # For sGrid/sview.Raster, it often is a NumPy masked array.
        if streams_raster_view is not None:
            streams_path = os.path.join(output_dir, 'streams_gee.tif')
            with rasterio.open(actual_dem_path_for_pysheds) as src_ds: # Get profile from original
                profile = src_ds.profile.copy()
                # Update profile for the output: 1 band, uint8
                profile.update(dtype=rasterio.uint8, count=1, compress='lzw', nodata=0) # Assuming 0 for no stream

            # sview.Raster can often be directly written or easily converted
            # It's typically a NumPy MaskedArray. We need to handle the mask.
            if hasattr(streams_raster_view, 'filled'):
                data_to_save = streams_raster_view.filled(0).astype(np.uint8) # Fill masked values with 0
            elif isinstance(streams_raster_view, np.ndarray):
                data_to_save = streams_raster_view.astype(np.uint8)
            else:
                print("Could not determine how to get savable NumPy array from streams_raster_view")
                data_to_save = None

            if data_to_save is not None:
                with rasterio.open(streams_path, 'w', **profile) as dst:
                    dst.write(data_to_save, 1)
                print(f"Streams raster saved to {streams_path}")
        else:
            print("streams_raster_view is None after trying grid.view('streams').")

    except KeyError:
        print("grid.view('streams') not accessible (KeyError).")
else:
    print("Skipping stream extraction as 'acc' RasterView was not obtained.")

print("Processing attempt finished.")

2025-05-17 18:10:29,332 - WARNING - CPLE_NotSupported in driver GTiff does not support open option DATA_NAME


Starting hydrological analysis with PySheds (sGrid refined) using: output_data_gee/gee_srtm_aoi.tif
Created grid object: <class 'pysheds.sgrid.sGrid'>
Loading DEM from output_data_gee/gee_srtm_aoi.tif...
DEM RasterView loaded. Type: <class 'pysheds.sview.Raster'>
Grid properties after read_raster: Shape: (1, 1), Affine: | 1.00, 0.00, 0.00|
| 0.00, 1.00, 0.00|
| 0.00, 0.00, 1.00|, Nodata: 0
Filling depressions...
Depressions filled. Returned type: <class 'pysheds.sview.Raster'>
Has 'flooded_dem' attribute on grid? False


TypeError: data must be a Raster instance

In [15]:
import pysheds
from pysheds.grid import Grid
import numpy as np
import rasterio
import os
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

output_dir = 'output_data_gee/'
os.makedirs(output_dir, exist_ok=True)
actual_dem_path_for_pysheds = os.path.join(output_dir, 'gee_srtm_aoi.tif')

if not os.path.exists(actual_dem_path_for_pysheds): # Corrected variable name
    logger.critical(f"CRITICAL: DEM file not found at {actual_dem_path_for_pysheds}")
    # exit()

logger.info(f"Starting hydrological analysis with PySheds (sGrid - accumulation fix) using: {actual_dem_path_for_pysheds}")

grid = Grid()
logger.debug(f"Initial grid.nodata: {grid.nodata} (type: {type(grid.nodata)}), grid.dtype: {getattr(grid, 'dtype', 'N/A')}")

logger.info(f"Loading DEM from {actual_dem_path_for_pysheds}...")
dem_raster_view = grid.read_raster(actual_dem_path_for_pysheds, data_name='dem_initial')
logger.info(f"DEM RasterView loaded. Type: {type(dem_raster_view)}")
logger.debug(f"dem_raster_view.dtype: {dem_raster_view.dtype}, dem_raster_view.nodata (direct): {getattr(dem_raster_view, 'nodata', 'N/A')}, from metadata: {dem_raster_view.metadata.get('nodata')}")

if hasattr(dem_raster_view, 'nodata'):
    grid.nodata = dem_raster_view.nodata
    logger.debug(f"Set grid.nodata from dem_raster_view.nodata: {grid.nodata} (type: {type(grid.nodata)})")
else:
    grid.nodata = np.int16(dem_raster_view.metadata.get('nodata', 0))
    logger.debug(f"Set grid.nodata from dem_raster_view.metadata (defaulting to 0 if None): {grid.nodata} (type: {type(grid.nodata)})")

if hasattr(dem_raster_view, 'dtype'):
    grid.dtype = dem_raster_view.dtype
    logger.debug(f"Set grid.dtype from dem_raster_view.dtype: {grid.dtype}")

logger.info(f"Grid properties after read_raster & updates: Shape: {grid.shape}, Affine: {grid.affine}, Nodata: {grid.nodata}, CRS: {grid.crs}, Dtype: {grid.dtype}")

logger.info("Filling depressions...")
flooded_dem_raster_view = grid.fill_depressions(dem=dem_raster_view, out_name='flooded_dem')
logger.info(f"Depressions filled. Returned type: {type(flooded_dem_raster_view)}")
logger.debug(f"flooded_dem_raster_view.dtype: {flooded_dem_raster_view.dtype}, .nodata: {getattr(flooded_dem_raster_view, 'nodata', 'N/A')}, metadata nodata: {flooded_dem_raster_view.metadata.get('nodata')}")

logger.info("Calculating flow direction...")
fdir_raster_view = None
if flooded_dem_raster_view is not None:
    grid_nodata_backup = grid.nodata
    grid_dtype_backup = grid.dtype
    flowdir_output_nodata_typed = np.uint8(0)
    flowdir_output_dtype = np.uint8 # Expect uint8, but log showed int64, this might need adjustment if error reappears
    
    logger.debug(f"Temporarily setting grid.nodata to {flowdir_output_nodata_typed} (type {type(flowdir_output_nodata_typed)}) and grid.dtype to {flowdir_output_dtype} for flowdir.")
    grid.nodata = flowdir_output_nodata_typed
    grid.dtype = flowdir_output_dtype
    try:
        fdir_raster_view = grid.flowdir(dem=flooded_dem_raster_view, out_name='fdir', nodata_out=flowdir_output_nodata_typed)
        logger.info(f"Flow direction calculated. Returned type: {type(fdir_raster_view)}")
        logger.debug(f"fdir_raster_view.dtype: {fdir_raster_view.dtype}, .nodata: {getattr(fdir_raster_view, 'nodata', 'N/A')}, metadata nodata: {fdir_raster_view.metadata.get('nodata')}")
    except Exception as e:
        logger.error(f"Error during grid.flowdir: {e}", exc_info=True)
    finally:
        grid.nodata = grid_nodata_backup
        grid.dtype = grid_dtype_backup
        logger.debug(f"Restored grid.nodata to: {grid.nodata} (type: {type(grid.nodata)}), grid.dtype to: {grid.dtype}")
else:
    logger.warning("Skipping flow direction as flooded_dem_raster_view is None.")


logger.info("Calculating flow accumulation...")
acc_raster_view = None
if fdir_raster_view is not None:
    # Accumulation output is often float32 or int64.
    # The grid.nodata and grid.dtype should be compatible with the *input* fdir for this operation.
    # The previous restoration of grid.nodata/dtype to DEM's original should be fine.
    logger.debug(f"grid.nodata before accumulation: {grid.nodata} (type: {type(grid.nodata)}), grid.dtype: {grid.dtype}")
    logger.debug(f"fdir_raster_view.nodata before accumulation: {getattr(fdir_raster_view, 'nodata', 'N/A')} (type: {type(getattr(fdir_raster_view, 'nodata', 'N/A'))}), fdir_raster_view.dtype: {fdir_raster_view.dtype}")

    try:
        # CORRECTED: Pass fdir_raster_view to the 'fdir' parameter.
        # 'out_name' registers 'acc' internally.
        # accumulation may or may not return an sview.Raster. We get it via get_data.
        grid.accumulation(fdir=fdir_raster_view, out_name='acc') # No 'data=' keyword
        logger.info(f"Flow accumulation calculated.")
        acc_raster_view = grid.get_data('acc', return_sview=True)
        logger.info(f"Accessed 'acc' via grid.get_data(), type: {type(acc_raster_view)}")
        logger.debug(f"acc_raster_view.dtype: {acc_raster_view.dtype}, .nodata: {getattr(acc_raster_view, 'nodata', 'N/A')}, metadata nodata: {acc_raster_view.metadata.get('nodata')}")
    except Exception as e:
        logger.error(f"Error during/after grid.accumulation: {e}", exc_info=True)
else:
    logger.warning("Skipping flow accumulation as fdir_raster_view was not created or is None.")


logger.info("Extracting stream network with threshold: 1000...")
streams_raster_view = None
if acc_raster_view is not None:
    grid_nodata_backup = grid.nodata
    grid_dtype_backup = grid.dtype
    streams_output_nodata_typed = np.uint8(0)
    streams_output_dtype = np.uint8

    logger.debug(f"Temporarily setting grid.nodata to {streams_output_nodata_typed} and grid.dtype to {streams_output_dtype} for streams.")
    grid.nodata = streams_output_nodata_typed
    grid.dtype = streams_output_dtype
    try:
        # CORRECTED: Pass fdir_raster_view and acc_raster_view to 'fdir' and 'acc' parameters.
        grid.extract_river_network(fdir=fdir_raster_view, acc=acc_raster_view, threshold=1000, out_name='streams')
        logger.info(f"Stream network extracted.")
        streams_raster_view = grid.get_data('streams', return_sview=True)
        logger.info(f"Accessed 'streams' via grid.get_data(), type: {type(streams_raster_view)}")
        logger.debug(f"streams_raster_view.dtype: {streams_raster_view.dtype}, .nodata: {getattr(streams_raster_view, 'nodata', 'N/A')}, metadata nodata: {streams_raster_view.metadata.get('nodata')}")

        if streams_raster_view is not None:
            streams_path = os.path.join(output_dir, 'streams_gee.tif')
            # Use grid.shape, grid.affine, grid.crs from the *initial* DEM load for consistent georeferencing.
            profile = {
                'driver': 'GTiff', 'dtype': rasterio.uint8, 'nodata': streams_output_nodata_typed.item(),
                'width': grid.shape[1], 'height': grid.shape[0], 'count': 1,
                'crs': grid.crs, 'transform': grid.affine, 'compress': 'lzw'
            }
            if hasattr(streams_raster_view, 'filled'): # sview.Raster is often a MaskedArray
                data_to_save = streams_raster_view.filled(streams_output_nodata_typed.item()).astype(np.uint8)
            elif isinstance(streams_raster_view, np.ndarray):
                data_to_save = streams_raster_view.astype(np.uint8)
            else:
                logger.error(f"Could not convert streams_raster_view to a savable NumPy array.")
                data_to_save = None

            if data_to_save is not None:
                with rasterio.open(streams_path, 'w', **profile) as dst:
                    dst.write(data_to_save, 1)
                logger.info(f"Streams raster saved to {streams_path}")
    except Exception as e:
        logger.error(f"Error obtaining or saving 'streams' data: {e}", exc_info=True)
    finally:
        grid.nodata = grid_nodata_backup
        grid.dtype = grid_dtype_backup
        logger.debug(f"Restored grid.nodata to: {grid.nodata}, grid.dtype to: {grid.dtype}")
else:
    logger.warning("Skipping stream extraction as acc_raster_view was not obtained or is None.")

logger.info("Processing attempt finished.")

2025-05-17 18:31:36,767 - INFO - Starting hydrological analysis with PySheds (sGrid - accumulation fix) using: output_data_gee/gee_srtm_aoi.tif
2025-05-17 18:31:36,772 - DEBUG - Initial grid.nodata: 0 (type: <class 'int'>), grid.dtype: N/A
2025-05-17 18:31:36,775 - INFO - Loading DEM from output_data_gee/gee_srtm_aoi.tif...
2025-05-17 18:31:36,781 - WARNING - CPLE_NotSupported in driver GTiff does not support open option DATA_NAME


2025-05-17 18:31:37,134 - INFO - DEM RasterView loaded. Type: <class 'pysheds.sview.Raster'>
2025-05-17 18:31:37,134 - DEBUG - dem_raster_view.dtype: int16, dem_raster_view.nodata (direct): -32768, from metadata: None
2025-05-17 18:31:37,134 - DEBUG - Set grid.nodata from dem_raster_view.nodata: -32768 (type: <class 'numpy.int16'>)
2025-05-17 18:31:37,134 - DEBUG - Set grid.dtype from dem_raster_view.dtype: int16
2025-05-17 18:31:37,134 - INFO - Grid properties after read_raster & updates: Shape: (1, 1), Affine: | 1.00, 0.00, 0.00|
| 0.00, 1.00, 0.00|
| 0.00, 0.00, 1.00|, Nodata: -32768, CRS: epsg:4326, Dtype: int16
2025-05-17 18:31:37,143 - INFO - Filling depressions...
2025-05-17 18:32:23,134 - INFO - Depressions filled. Returned type: <class 'pysheds.sview.Raster'>
2025-05-17 18:32:23,136 - DEBUG - flooded_dem_raster_view.dtype: int16, .nodata: -32768, metadata nodata: None
2025-05-17 18:32:23,138 - INFO - Calculating flow direction...
2025-05-17 18:32:23,140 - DEBUG - Temporarily s

In [16]:
import pysheds
from pysheds.grid import Grid
import numpy as np
import rasterio
import os
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

output_dir = 'output_data_gee/'
os.makedirs(output_dir, exist_ok=True)
actual_dem_path_for_pysheds = os.path.join(output_dir, 'gee_srtm_aoi.tif')

if not os.path.exists(actual_dem_path_for_pysheds):
    logger.critical(f"CRITICAL: DEM file not found at {actual_dem_path_for_pysheds}")
    # exit()

logger.info(f"Starting hydrological analysis with PySheds (sGrid - accumulation nodata fix) using: {actual_dem_path_for_pysheds}")

grid = Grid()
logger.debug(f"Initial grid.nodata: {grid.nodata} (type: {type(grid.nodata)}), grid.dtype: {getattr(grid, 'dtype', 'N/A')}")

logger.info(f"Loading DEM from {actual_dem_path_for_pysheds}...")
dem_raster_view = grid.read_raster(actual_dem_path_for_pysheds, data_name='dem_initial')
logger.info(f"DEM RasterView loaded. Type: {type(dem_raster_view)}")
logger.debug(f"dem_raster_view.dtype: {dem_raster_view.dtype}, dem_raster_view.nodata (direct): {getattr(dem_raster_view, 'nodata', 'N/A')}, from metadata: {dem_raster_view.metadata.get('nodata')}")

if hasattr(dem_raster_view, 'nodata'):
    grid.nodata = dem_raster_view.nodata
    logger.debug(f"Set grid.nodata from dem_raster_view.nodata: {grid.nodata} (type: {type(grid.nodata)})")
else:
    grid.nodata = np.int16(dem_raster_view.metadata.get('nodata', 0))
    logger.debug(f"Set grid.nodata from dem_raster_view.metadata: {grid.nodata} (type: {type(grid.nodata)})")

if hasattr(dem_raster_view, 'dtype'):
    grid.dtype = dem_raster_view.dtype
    logger.debug(f"Set grid.dtype from dem_raster_view.dtype: {grid.dtype}")

logger.info(f"Grid properties after read_raster & updates: Shape: {grid.shape}, Affine: {grid.affine}, Nodata: {grid.nodata}, CRS: {grid.crs}, Dtype: {grid.dtype}")

logger.info("Filling depressions...")
flooded_dem_raster_view = grid.fill_depressions(dem=dem_raster_view, out_name='flooded_dem')
logger.info(f"Depressions filled. Returned type: {type(flooded_dem_raster_view)}")
logger.debug(f"flooded_dem_raster_view.dtype: {flooded_dem_raster_view.dtype}, .nodata: {getattr(flooded_dem_raster_view, 'nodata', 'N/A')}, metadata nodata: {flooded_dem_raster_view.metadata.get('nodata')}")

logger.info("Calculating flow direction...")
fdir_raster_view = None
if flooded_dem_raster_view is not None:
    grid_nodata_backup_fdir = grid.nodata
    grid_dtype_backup_fdir = grid.dtype
    flowdir_output_nodata_typed = np.uint8(0)
    flowdir_output_dtype = np.uint8 # Though output was int64, let's keep this for consistency with typical D8. sGrid might override.
    
    logger.debug(f"Temporarily setting grid.nodata to {flowdir_output_nodata_typed} (type {type(flowdir_output_nodata_typed)}) and grid.dtype to {flowdir_output_dtype} for flowdir.")
    grid.nodata = flowdir_output_nodata_typed
    grid.dtype = flowdir_output_dtype
    try:
        fdir_raster_view = grid.flowdir(dem=flooded_dem_raster_view, out_name='fdir', nodata_out=flowdir_output_nodata_typed)
        logger.info(f"Flow direction calculated. Returned type: {type(fdir_raster_view)}")
        logger.debug(f"fdir_raster_view.dtype: {fdir_raster_view.dtype}, .nodata: {getattr(fdir_raster_view, 'nodata', 'N/A')}, metadata nodata: {fdir_raster_view.metadata.get('nodata')}")
    except Exception as e:
        logger.error(f"Error during grid.flowdir: {e}", exc_info=True)
    finally:
        grid.nodata = grid_nodata_backup_fdir
        grid.dtype = grid_dtype_backup_fdir
        logger.debug(f"Restored grid.nodata to: {grid.nodata} (type: {type(grid.nodata)}), grid.dtype to: {grid.dtype}")
else:
    logger.warning("Skipping flow direction as flooded_dem_raster_view is None.")

logger.info("Calculating flow accumulation...")
acc_raster_view = None
if fdir_raster_view is not None:
    grid_nodata_backup_acc = grid.nodata
    grid_dtype_backup_acc = grid.dtype

    # Accumulation output is typically int64 or float32/64.
    # Let's set grid.nodata to 0 and grid.dtype to int64 temporarily.
    acc_output_nodata_typed = np.int64(0) # Or a large negative if that's more standard for int acc nodata
    acc_output_dtype = np.int64

    logger.debug(f"Temporarily setting grid.nodata to {acc_output_nodata_typed} (type {type(acc_output_nodata_typed)}) and grid.dtype to {acc_output_dtype} for accumulation.")
    grid.nodata = acc_output_nodata_typed
    grid.dtype = acc_output_dtype
    
    # Also check if `accumulation` has a `nodata_out` or similar parameter
    # Based on sGrid source, it doesn't seem to have an explicit `nodata_out`.
    # It will use the grid's current nodata setting for its output viewfinder.

    try:
        grid.accumulation(fdir=fdir_raster_view, out_name='acc')
        logger.info(f"Flow accumulation calculated.")
        acc_raster_view = grid.get_data('acc', return_sview=True)
        logger.info(f"Accessed 'acc' via grid.get_data(), type: {type(acc_raster_view)}")
        logger.debug(f"acc_raster_view.dtype: {acc_raster_view.dtype}, .nodata: {getattr(acc_raster_view, 'nodata', 'N/A')}, metadata nodata: {acc_raster_view.metadata.get('nodata')}")
    except Exception as e:
        logger.error(f"Error during/after grid.accumulation: {e}", exc_info=True)
    finally:
        grid.nodata = grid_nodata_backup_acc
        grid.dtype = grid_dtype_backup_acc
        logger.debug(f"Restored grid.nodata to: {grid.nodata} (type: {type(grid.nodata)}), grid.dtype to: {grid.dtype}")
else:
    logger.warning("Skipping flow accumulation as fdir_raster_view was not created or is None.")

# ... (Rest of the script for stream extraction and saving, applying similar temporary nodata/dtype if needed) ...
logger.info("Extracting stream network with threshold: 1000...")
streams_raster_view = None
if acc_raster_view is not None:
    grid_nodata_backup_streams = grid.nodata
    grid_dtype_backup_streams = grid.dtype
    streams_output_nodata_typed = np.uint8(0)
    streams_output_dtype = np.uint8

    logger.debug(f"Temporarily setting grid.nodata to {streams_output_nodata_typed} and grid.dtype to {streams_output_dtype} for streams.")
    grid.nodata = streams_output_nodata_typed
    grid.dtype = streams_output_dtype
    try:
        grid.extract_river_network(fdir=fdir_raster_view, acc=acc_raster_view, threshold=1000, out_name='streams')
        logger.info(f"Stream network extracted.")
        streams_raster_view = grid.get_data('streams', return_sview=True)
        logger.info(f"Accessed 'streams' via grid.get_data(), type: {type(streams_raster_view)}")
        logger.debug(f"streams_raster_view.dtype: {streams_raster_view.dtype}, .nodata: {getattr(streams_raster_view, 'nodata', 'N/A')}, metadata nodata: {streams_raster_view.metadata.get('nodata')}")

        if streams_raster_view is not None:
            streams_path = os.path.join(output_dir, 'streams_gee.tif')
            profile = {
                'driver': 'GTiff', 'dtype': rasterio.uint8, 'nodata': streams_output_nodata_typed.item(),
                'width': grid.shape[1], 'height': grid.shape[0], 'count': 1,
                'crs': grid.crs, 'transform': grid.affine, 'compress': 'lzw'
            }
            if hasattr(streams_raster_view, 'filled'):
                data_to_save = streams_raster_view.filled(streams_output_nodata_typed.item()).astype(np.uint8)
            elif isinstance(streams_raster_view, np.ndarray):
                data_to_save = streams_raster_view.astype(np.uint8)
            else:
                logger.error(f"Could not convert streams_raster_view to a savable NumPy array.")
                data_to_save = None

            if data_to_save is not None:
                with rasterio.open(streams_path, 'w', **profile) as dst:
                    dst.write(data_to_save, 1)
                logger.info(f"Streams raster saved to {streams_path}")
    except Exception as e:
        logger.error(f"Error obtaining or saving 'streams' data: {e}", exc_info=True)
    finally:
        grid.nodata = grid_nodata_backup_streams
        grid.dtype = grid_dtype_backup_streams
        logger.debug(f"Restored grid.nodata to: {grid.nodata}, grid.dtype to: {grid.dtype}")
else:
    logger.warning("Skipping stream extraction as acc_raster_view was not obtained or is None.")

logger.info("Processing attempt finished.")

2025-05-17 18:33:54,847 - INFO - Starting hydrological analysis with PySheds (sGrid - accumulation nodata fix) using: output_data_gee/gee_srtm_aoi.tif
2025-05-17 18:33:54,850 - DEBUG - Initial grid.nodata: 0 (type: <class 'int'>), grid.dtype: N/A
2025-05-17 18:33:54,850 - INFO - Loading DEM from output_data_gee/gee_srtm_aoi.tif...
2025-05-17 18:33:54,852 - WARNING - CPLE_NotSupported in driver GTiff does not support open option DATA_NAME
2025-05-17 18:33:55,053 - INFO - DEM RasterView loaded. Type: <class 'pysheds.sview.Raster'>
2025-05-17 18:33:55,055 - DEBUG - dem_raster_view.dtype: int16, dem_raster_view.nodata (direct): -32768, from metadata: None
2025-05-17 18:33:55,055 - DEBUG - Set grid.nodata from dem_raster_view.nodata: -32768 (type: <class 'numpy.int16'>)
2025-05-17 18:33:55,055 - DEBUG - Set grid.dtype from dem_raster_view.dtype: int16
2025-05-17 18:33:55,055 - INFO - Grid properties after read_raster & updates: Shape: (1, 1), Affine: | 1.00, 0.00, 0.00|
| 0.00, 1.00, 0.00|
